## Gate cutting to reduce SWAP overhead

In [1]:
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import PauliList

Specify a circuit and some observables

In [2]:
circuit = EfficientSU2(num_qubits=4, entanglement="circular").decompose()
circuit.assign_parameters([0.4] * len(circuit.parameters), inplace=True)
observables = PauliList(["ZZII", "IZZI", "IIZZ", "XIXI", "ZIZZ", "IXIX"])

Cut the long-distant gates by using ``cut_gates`` and specifying their indices. Generate the subexperiments and their weights using ``generate_cutting_experiments``.

In [3]:
from circuit_knitting.cutting import cut_gates, generate_cutting_experiments

# Find the indices of the distant gates
cut_indices = [
    i
    for i, instruction in enumerate(circuit.data)
    if {circuit.find_bit(q)[0] for q in instruction.qubits} == {0, 3}
]

# Cut the long-distant gates and generate the subexperiments
qpd_circuit, bases = cut_gates(circuit, cut_indices)
subexperiments, weights = generate_cutting_experiments(qpd_circuit, observables, np.inf)

# Get the number of clbits associated with QPD measurements
num_decomps = len(subexperiments[0].cregs[0])

Run the subexperiments for each partition using the Qiskit Aer primitives.

In [4]:
from qiskit_aer.primitives import Sampler

# Set up Qiskit Aer Sampler primitive.
sampler = Sampler(run_options={"shots": 2**12})

# Retrieve the quasi-probability distributions for each subexperiment
quasi_dists = sampler.run(subexperiments).result().quasi_dists

To run using Qiskit Runtime primitives, rather than Qiskit Aer, replace the code above with this commented block.

In [5]:
# from qiskit_ibm_runtime import Session, Options, Sampler

# with Session(backend="<BACKEND_NAME>") as session:
#    options = Options()
#    options.execution.shots = 2**12
#    sampler = Sampler(options=options)
#    quasi_dists = sampler.run(subexperiments).result().quasi_dists

#    session.close()

Reconstruct the full expectation value, given the results of cutting

In [6]:
from circuit_knitting.cutting import reconstruct_expectation_values

simulated_expvals = reconstruct_expectation_values(
    observables,
    weights,
    quasi_dists,
    num_decomps,
)
simulated_expvals

[0.5159912109374982,
 0.5632934570312489,
 0.351806640624999,
 -0.23968505859374928,
 0.23175048828124972,
 -0.21936035156249925]